In [1]:
from binance import Client
import sqlalchemy
import numpy as np
import pandas as pd
import time
import ta
from datetime import datetime
from sklearn import preprocessing
import math
pd.options.mode.chained_assignment = None  # default='warn'

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
%run Binance_keys.py # you will need to generate your own python keys

In [ ]:
client = Client(api_key,api_secret)

In [ ]:
engine = sqlalchemy.create_engine('sqlite:///C:\\User\\')

In [ ]:
chart = '4h'
interval = Client.KLINE_INTERVAL_4HOUR
lookback = '30 day ago UTC'
stakeUSD = 150

In [ ]:
def get_best_performers(klines = {}, chart = '1m', timeframe = '30 minute ago'):
    client = Client()
    info = client.get_exchange_info()
    symbols =  [x['symbol'] for x in info['symbols']]
    exclude = ['UP', 'DOWN', 'BEAR', 'BULL', 'XMR', 'ZEC', 'DASH']
    non_lev = [symbol for symbol in symbols if all(excludes not in symbol for excludes in exclude)]
    relevant = [symbol for symbol in non_lev if symbol.endswith('USDT')]
    for symbol in tqdm(relevant):
        klines[symbol] = client.get_historical_klines(symbol, chart, '30 minute ago')

    returns, symbols = [], [] 

    for symbol in relevant:
        if len(klines[symbol]) > 0:
            cumret = (pd.DataFrame(klines[symbol])[4].astype(float).pct_change() + 1).prod()-1
            returns.append(cumret)
            symbols.append(symbol) 
    retdf = pd.DataFrame(returns, index=symbols, columns=['ret'])
    top_10 = retdf.ret.nlargest(10)
    top_10_symbols = top_10.index.tolist()
    return top_10_symbols


In [ ]:
def get_top_symbol():
    all_pairs = pd.DataFrame(client.get_ticker())
    relev = all_pairs[all_pairs.symbol.str.contains('USDT')]
    relev['priceChangePercent'] = relev['priceChangePercent'].astype(float)
    relev['quoteVolume'] = relev['quoteVolume'].astype(float)
    non_lev = relev[~((relev.symbol.str.contains('UP')) | (relev.symbol.str.contains('DOWN')) | (relev.symbol.str.contains('ZEC')) | (relev.symbol.str.contains('ZEN')) | (relev.symbol.str.startswith('USDT')) | (relev.symbol.str.contains('BUSD')) | (relev.symbol.str.contains('DASH')) | (relev.symbol.str.contains('BTC')))]
    non_lev = non_lev[non_lev['quoteVolume'] >= 10000000]
    all_pairs_sorted = non_lev.sort_values(['priceChangePercent', 'quoteVolume'], ascending=[False, False])
    top_symbol = all_pairs_sorted.symbol.values.tolist()
    return top_symbol

In [ ]:
def get_asset_url(asset):
    reversedAsset = asset[::-1]
    _asset = reversedAsset[:4] + '_' + reversedAsset[4:]
    tradingAsset = _asset[::-1]
    url = 'https://www.binance.com/en/trade/' + str(tradingAsset)
    return url

In [ ]:
def getminutedata(symbol, interval, lookback): # symbol, time intercal (mintues), lookback (how far back)
    frame = pd.DataFrame(client.get_historical_klines(symbol,
                                                     interval,
                                                     lookback + ' min ago UTC'))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    return frame

In [ ]:
def gethourdata(symbol, interval, lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol,
                                                     interval,
                                                     lookback))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    return frame

In [ ]:
def applytechnicals(frame):
    frame['%K'] = ta.momentum.stoch(frame.High, frame.Low, frame.Close, window = 14,
                                smooth_window=3)
    frame['%D'] = frame['%K'].rolling(3).mean()
    frame['rsi'] = ta.momentum.rsi(frame.Close, window= 14) # was 5 but 14 more conservative, better trading signals
    frame['sma28'] = frame.Close.rolling(28).mean()
    frame['macd'] = ta.trend.macd_diff(frame.Close)
    frame.dropna(inplace=True)
    return frame

In [ ]:
def get_lot_size(symbol):
    x = client.get_symbol_info(symbol)
    step_size = float(x['filters'][2]['stepSize'])
    decimal_places = abs(int(f'{step_size:e}'.split('e')[-1]))
    return decimal_places

In [ ]:
def round_decimals_down(number, decimal_places):
    if not isinstance(decimal_places, int):
        raise TypeError("decimal places must be an integer")
    elif decimal_places < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimal_places == 0:
        return math.floor(number)
    factor = 10 ** decimal_places
    return math.floor(number * factor) / factor

In [ ]:
def strategy(buy_amt, SL=0.980, Target = 1.025, open_position = False):  
    for symbol in get_top_symbol():
        time.sleep(2)
        try:
            asset = symbol
            df = gethourdata(asset, interval, lookback)
            df = applytechnicals(df)
            decimal_places = get_lot_size(asset)
            print(str(datetime.now()) + ': Waiting for buy signal in binance trading pair: ' + str(asset), end='\r')
            qty = round(buy_amt/df.Close.iloc[-1], decimal_places)
        except Exception:
            pass

        #if ((df.Close.pct_change() + 1).cumprod()).iloc[-1] > 1:  # buying condition if the percentage increasing by >1 in 1min
        if len(df.index) != 0 and df.rsi[-1] < 30 and df.macd.iloc[-1] > df.macd.iloc[-2]:
            order = client.create_order(symbol = asset,
                                       side = 'BUY',
                                       type = 'MARKET',
                                        quantity = qty)
            print(order)
            #buyprice = float(order['fills'][0]['price'])
            open_position = True
            stopLoss = abs(round((1-SL)*100, 2))

            commission_array = []
            buy_price_array = []
            qty_array = []
            i = 0

            while i <= len(order['fills'])-1:
                commission = float(order['fills'][i]['commission'])
                buy_price = float(order['fills'][i]['price'])
                qty = float(order['fills'][i]['qty'])
                commission_array.append(commission)
                buy_price_array.append(buy_price)
                qty_array.append(qty)
                i = i + 1

            total_commission = sum(commission_array)
            average_buy_price = sum(buy_price_array) / len(buy_price_array)
            qty = sum(qty_array)

            total_buy_amount = average_buy_price * qty

            for k,v in order.items():
                if k == 'clientOrderId':
                    clientOrderId = v
                if k == 'symbol':
                    tradingPair = v
                if k == 'type':
                    spotType = v
                if k == 'side':
                    buyOrSell = v
                if k == 'fills':
                    transaction = v[0]
                    transaction['timestamp'] = datetime.now()
                    transaction['tradingPair'] = tradingPair
                    transaction['spotType'] = spotType
                    transaction['buyOrSell'] = buyOrSell
                    transaction['clientOrderId'] = clientOrderId
                    transaction['profit'] = 0
                    transaction['rsi'] = float(df.rsi.iloc[-1])
                    transaction['sma28'] = float(df.sma28.iloc[-1])
                    transaction['macd'] = float(df.macd.iloc[-1])
                    transaction['roi'] = 0
                    transaction['chart'] = chart
                    transaction['totalAmount'] = float(total_buy_amount)
                    transaction['strategy'] = 'MACD Increase, RSI < 30 with a TSL  ' + str(abs(stopLoss)) + '%'
                    transaction_array.append(transaction)
                    #print(transaction)

            trailing_stop_loss = average_buy_price * SL # set global variable starting at the buy price * the stop loss


        #SELL CONDITIONS BELOW:


        while open_position:
            try:
                time.sleep(2)
                df = gethourdata(asset, interval, lookback)
                df = applytechnicals(df)
            except:
                print('Somethings gone wrong, script continues in 1m... unless youve stopped it automatically...')
                time.sleep(61)
                df = gethourdata(asset, interval, lookback)
                df = applytechnicals(df)

            if (df.Close.iloc[-1] * SL) > trailing_stop_loss:
                trailing_stop_loss = df.Close.iloc[-1] * SL  # create new trailing stop loss variable     

            tradingFee = len(order['fills']) * 0.001
            profit = ((trailing_stop_loss * qty) - (average_buy_price * qty)) - (average_buy_price * qty * tradingFee)
            roi = round(((trailing_stop_loss-average_buy_price)/average_buy_price)*100, 2)
            
            print(f'Time: ' + str(datetime.now()))
            print(f'Open Position in: ' + str(asset))
            print(f'Binance URL: ' + str(get_asset_url(asset)))
            #print(f'TSL Profit: $'+ str(round(profit, 2)))
            print(f'TSL ROI: '+ str(roi) + '%')
            print(f'Close ROI: ' + str(round(((df.Close[-1]-average_buy_price)/average_buy_price)*100, 2)) + '%')
            print(f'Current Close Price: $' + str(df.Close.iloc[-1]))
            print(f'Buy Price: $' + str(average_buy_price))
            print(f'Trailing Stop Loss: $' + str(trailing_stop_loss))
            print('----------------------------------------')
            time.sleep(1)
            if ( df.Close[-1] <= trailing_stop_loss ): #or (df.Close[-1] >= average_buy_price * Target): # sell only on trailing stop loss


                    order = client.create_order(symbol = asset,
                                    side = 'SELL',
                                    type = 'MARKET',
                                    quantity = qty)

                    commission_array = []
                    sell_price_array = []
                    qty_array = []
                    i = 0

                    while i <= len(order['fills'])-1:
                        commission = float(order['fills'][i]['commission'])
                        sell_price = float(order['fills'][i]['price'])
                        qty = float(order['fills'][i]['qty'])
                        commission_array.append(commission)
                        sell_price_array.append(sell_price)
                        qty_array.append(qty)
                        i = i + 1

                    total_commission = sum(commission_array)
                    average_sell_price = sum(sell_price_array) / len(sell_price_array)
                    qty = sum(qty_array)

                    total_sell_amount = average_sell_price * qty

                    roi = round(((total_sell_amount-total_buy_amount)/total_buy_amount)*100, 2)

                    for k,v in order.items():
                        if k == 'clientOrderId':
                            clientOrderId = v
                        if k == 'symbol':
                            tradingPair = v
                        if k == 'type':
                            spotType = v
                        if k == 'side':
                            buyOrSell = v
                        if k == 'fills':
                            transaction = v[0]
                            transaction['timestamp'] = datetime.now()
                            transaction['tradingPair'] = tradingPair
                            transaction['spotType'] = spotType
                            transaction['buyOrSell'] = buyOrSell
                            transaction['clientOrderId'] = clientOrderId
                            transaction['profit'] = profit
                            transaction['rsi'] = float(df.rsi.iloc[-1])
                            transaction['sma28'] = float(df.sma28.iloc[-1])
                            transaction['macd'] = float(df.macd.iloc[-1])
                            transaction['roi'] = float(roi)
                            transaction['chart'] = chart
                            transaction['totalAmount'] = float(total_sell_amount)
                            transaction['strategy'] = 'MACD Increase, RSI < 30 with a TSL  ' + str(abs(stopLoss)) + '%'
                            transaction_array.append(transaction)
                            print(transaction_array)
                    return "Finished"

In [ ]:
while True:
    transaction_array = []
    strategy(stakeUSD)
    botTrades = pd.DataFrame(transaction_array)
    botTrades.to_sql('bot_transactions', engine, if_exists='append', index=True)
    time.sleep(0.5)